In [13]:
import cv2
import glob
from ultralytics import YOLO
import os

In [ ]:
def process_frames_folder(folder_path):
    print(f"Loading the Model...")

    model = YOLO('yolov8n.pt')

    image_pattern = os.path.join(folder_path, '*.jpeg')
    frame_files = glob.glob(image_pattern)

    frame_files.sort()

    if not frame_files:
        print(f"No Images in the folder: {folder_path}")
        return

    print(f"Found images.. now processing in {folder_path}")

    print("\nDetection Results (CSV Format)")
    print("frame_filename,track_id,class_id,class_name,confidence,x_min,y_min,x_max,y_max")

    track_results = model.track(source=frame_files, persist=True, tracker='bytetrack.yaml', verbose=False)

    for result in track_results:
        frame_filename = None
        if hasattr(result, 'orig_img_path') and result.orig_img_path:
            frame_filename = os.path.basename(result.orig_img_path)
        elif hasattr(result, 'path') and result.path:
            frame_filename = os.path.basename(result.path)
        else:
            frame_filename = "unknown_frame"

        try:
            boxes = result.boxes.xyxy.gpu().numpy()
        except Exception:
            try:
                boxes = result.boxes.gpu().numpy()[:, :4]
            except Exception:
                boxes = []

        try:
            confidences = result.boxes.conf.gpu().numpy()
        except Exception:
            confidences = [0.0] * len(boxes)

        try:
            class_ids = result.boxes.cls.gpu().numpy().astype(int)
        except Exception:
            class_ids = [-1] * len(boxes)

        track_ids = None

        if hasattr(result.boxes, 'id'):
            try:
                track_ids = result.boxes.id.gpu().numpy().astype(int)
            except Exception:
                track_ids = None

        if track_ids is None and hasattr(result.boxes, 'track_id'):
            try:
                track_ids = result.boxes.track_id.gpu().numpy().astype(int)
            except Exception:
                track_ids = None

        if track_ids is None and hasattr(result.boxes, 'data'):
            try:
                data = result.boxes.data.gpu().numpy()
                # If data has >=7 columns, last column often contains track id
                if data.ndim == 2 and data.shape[1] >= 7:
                    track_ids = data[:, -1].astype(int)
            except Exception:
                track_ids = None

        if track_ids is None:
            track_ids = [-1] * len(boxes)

        for i in range(len(boxes)):
            box = boxes[i]
            x_min, y_min, x_max, y_max = box
            conf = float(confidences[i]) if i < len(confidences) else 0.0
            cls_id = int(class_ids[i]) if i < len(class_ids) else -1
            tid = int(track_ids[i]) if i < len(track_ids) else -1

            class_name = model.names.get(cls_id, str(cls_id)) if hasattr(model, 'names') else str(cls_id)

            if class_name in ['car', 'truck', 'bus', 'motorcycle']:
                print(
                    f"{frame_filename},"
                    f"{tid},"
                    f"{cls_id},"
                    f"{class_name},"
                    f"{conf:.4f},"
                    f"{int(x_min)},"
                    f"{int(y_min)},"
                    f"{int(x_max)},"
                    f"{int(y_max)}"
                )

    print("--- Processing complete ---")


In [18]:
if __name__ == "__main__":
    FRAMES_FOLDER = '/Users/yusse/Coding/knight_hacks25/Yolo_frames'

    if FRAMES_FOLDER == '/Users/yusse/Coding/knight_hacks25/Yolo_frames':
        print("="*50)
        print("ERROR: Please update the 'FRAMES_FOLDER' variable in this script")
        print(f"       to point to your directory of video frames.")
        print("="*50)
    else:
        if not os.path.isdir(FRAMES_FOLDER):
            print(f"Error: Folder not found at {FRAMES_FOLDER}")
        else:
            process_frames_folder(FRAMES_FOLDER)

SyntaxError: expected ':' (640311177.py, line 4)